In [31]:
from typing import Annotated
from dotenv import load_dotenv
load_dotenv()

from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


In [32]:
llm = init_chat_model('google_genai:gemini-2.0-flash')

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return { 'messages': [llm.invoke(state['messages'])]}

builder = StateGraph(State)
builder.add_node('chatbot_node', chatbot)
builder.add_edge(START, 'chatbot_node')
builder.add_edge('chatbot_node', END)

graph = builder.compile()


In [33]:
message = { "role": "user", 'content': "who is aamir khan"}
response = graph.invoke({'messages': [message]})
response['messages']

[HumanMessage(content='who is aamir khan', additional_kwargs={}, response_metadata={}, id='9438c76b-b45b-4a17-98fe-cd7d13e8f138'),
 AIMessage(content='Aamir Khan is a highly acclaimed and influential Indian actor, filmmaker, and television personality. He\'s considered one of the biggest and most successful stars in Hindi cinema (Bollywood). Here\'s a breakdown of why he\'s so well-known:\n\n*   **Acting:** He\'s known for his versatile acting skills and his dedication to choosing roles that are challenging and meaningful. He often immerses himself in his characters, undergoing significant physical transformations and spending considerable time researching and understanding the roles he plays.\n\n*   **Film Production:** Aamir Khan Productions, his production company, has been responsible for producing several critically acclaimed and commercially successful films that often tackle social issues and push boundaries in Indian cinema.\n\n*   **Social Activism:** Aamir Khan is known for u

In [34]:
state = None
while True:
    try:
        in_message = input('You: ')
        if in_message.lower() in {'quit', 'exit', 'stop'} :
            print('Goodbye!')
            break
        if state is None:
            state: State = {
                'messages': { 'role': 'user', 'content': in_message },
            }
        else:
            state['messages'].append({'role': 'user', 'content': in_message})
        state = graph.invoke(state)
        last_message = state['messages'][-1]
        print('Bot: ', last_message.content)
    except KeyboardInterrupt:
        print('Bot: Interrupted')
        break


Bot:  Amit Chaudhary is a **Research Scientist** at **Questt.ai**.

Based on his LinkedIn profile and other online presence, here's what's known about him:

*   **Role:** Research Scientist
*   **Company:** Questt.ai
*   **Education:** IIT Kanpur
*   **Skills/Expertise:** He appears to be involved in research and development related to AI and machine learning, likely within the context of Questt.ai's educational platform.

To get more specific information about his work, you might want to:

*   Look for his publications or presentations related to Questt.ai's technology.
*   Check Questt.ai's website or blog for mentions of his contributions.
*   Review his LinkedIn profile for more details on his projects and responsibilities.
Goodbye!
